## 의사결정 나무모델    

지도학습 알고리즘 (분류, 회귀)
- 레이블 (label), 즉 명시적인 정답이 있는 데이터가 주어진 상태에서 학습하는 머신 러닝 방식
- 대표적인 유형: 분류 (Classifification); 학습 데이터로 주어진 데이터의 피쳐와 레이블값(결정값, 클레스값)을 머신 러닝러닝 알고리즘으로 학습해 모델 생성, 새로운 데이터의 미지의 레이블을 예측)

직관적인 알고리즘 (이해 쉬움)
과대적합되기 쉬운 알고리즘 (트리 깊이 제한 필요)

* 분류 모델 (의사 결정 나무, 랜덤 포레스트, xgboost)
* 교차검증
* 평가 (분류)

In [1]:
# 기본적인 라이브러리 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# 데이터 생성
from sklearn.datasets import load_breast_cancer

def make_dataset():
    iris = load_breast_cancer()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('target', axis=1), df['target'], test_size=0.5, random_state=1004)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_dataset()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((284, 30), (285, 30), (284,), (285,))

In [3]:
# 타겟 확인 (2:1 정도의 값을 가지고 있는 것을 확인하였음)
y_train.value_counts()

1    190
0     94
Name: target, dtype: int64

## 의사결정나무

지도학습(분류)에서 가장 유용하게 사용되고 있는 기법 중 하나입니다.
트리의 루트(root)에서 시작해서 정보이득이 최대가 되는 특성으로 데이터를 나눕니다.
정보이득(information gain)이 최대가 되는 특성을 나누는 기준(불순도를 측정하는 기준)은 '지니'와 '엔트로피'가 사용됩니다.
데이터가 한 종류만 있다면 엔트로피/지니 불순도는 0에 가깝고, 서로 다른 데이터의 비율이 비슷하면 1에 가깝습니다.
정보이득(information gain)이 최대라는 것은 불순도를 최소화 하는 방향입니다. (1-불순도)

In [4]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0) #random state 지정하여 항상 값이 같도록 해야해
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred) #예측한 변수를 평가한다

0.9263157894736842

3. 의사결정나무 하이퍼파라미터
criterion (기본값 gini) : 불순도 지표 (또는 엔트로피 불순도 entropy)
max_depth (기본값 None) : 최대 한도 깊이
min_samples_split (기본값 2) : 자식 노드를 갖기 위한 최소한의 데이터 수
min_samples_leaf (기본값 1) : 리프 노드가 되기 위한 최소 샘플 수

In [5]:
# 의사결정나무 하이퍼파라미터
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(
    criterion = 'entropy', 
    max_depth = 7, # max_depth 튜닝에 따라 값이 달라질 것
    min_samples_split = 2, #적어도 2개 이상이 되어야 자식 노드가 생김
    min_samples_leaf=2, #역시 튜닝 값에 따라 값이 달라짐
    random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9228070175438596

## 랜덤포레스트
- 의사결정 나무가 여러 개 있는 것을 랜덤포레스트
- 지도 학습 알고리즘 (분류, 회귀)
- 의사결정나무의 앙상블
- 여러개의 의사결정 트리로 구성
- 성능이 좋음 (과대적합 가능성 낮음)
- 부트 스트랩 샘플링 (데이터셋 중복 허용)
- 최종 다수결 투표
- 앙상블 
> 배깅; 같은 알고리즘으로 여러 모델을 만들어 분류함 (랜덤 포레스트에서 채택하는 방식)
> 부스팅; 학습과 예측을 하면서 가중치 반영 (xgboost)

In [6]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0) #random state 0으로 고정
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9438596491228071

n_estimators (기본값 100) : 트리의 수 - 여러개의 트리가 합쳐져서 포르스트가 되기 때문
> 많을 수록 속도는 느려짐

criterion (기본값 gini) : 불순도 지표
max_depth (기본값 None) : 최대 한도 깊이
min_samples_split (기본값 2) : 자식 노드를 갖기 위한 최소한의 데이터 수
min_samples_leaf (기본값 1) : 리프 노드가 되기 위한 최소 샘플 수

In [8]:
# 랜덤포레스트 하이퍼파라미터
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0) 
#층과 depth를 조정 하면서 정확도를 조정함, 
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9473684210526315

## XGBoost

XGBoost 모델 더 알아보기

트리 앙상블 중 성능이 좋은 알고리즘
eXtreme Gradient Boosting를 줄여서 XGBoost라고 한다.
약한 학습기가 계속해서 업데이트를 하며 좋은 모델을 만들어 간다.
부스팅(앙상블) 기반의 알고리즘
캐글(글로벌 AI 경진대회)에서 뛰어난 성능을 보이면서 인기가 높아짐 (약한 알고리즘이 묶여서 자란다)

2. 하이퍼 파라미터의 종류
booster (기본값 gbtree) : 부스팅 알고리즘 (또는 dart, gblinear)
objective (기본값 binary:logistic) : 이진분류 (다중분류: multi:softmax)
max_depth (기본값 6) : 최대 한도 깊이
learning_rate (기본값 0.1) : 학습률
n_estimators (기본값 100) : 트리의 수
subsample (기본값 1) : 훈련 샘플 개수의 비율
colsample_bytree (기본값 1) : 특성 개수의 비율
n_jobs (기본값 1) : 사용 코어 수 (-1: 모든 코어를 다 사용)

In [9]:
# xgboost
from xgboost import XGBClassifierㅍ #불러오기
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)
#성능이 조금이라도 더 올랄감.. 워닝은 사실 있어도 상관 없음

/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(


0.9508771929824561

In [10]:
# xgboost 하이퍼파라미터
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss',
                      booster = 'gbtree',
                      objective = 'binary:logistic',
                      max_depth = 5,
                      learning_rate = 0.05,
                      n_estimators = 500,
                      subsample = 1, 
                      colsample_bytree = 1,
                      n_jobs = -1
                      
                      
# - booster(기본값 gbtree): 부스팅 알고리즘 (또는 dart, gblinear)
# - objective(기본값 binary:logistic): 이진분류 (다중분류: multi:softmax)
# - max_depth(기본값 6): 최대 한도 깊이
# - learning_rate(기본값 0.1): 학습률 - 기울기가 0인 점을 찾아가는 과정; x^2에서 미분하여 내려가는 과정
# - n_estimators(기본값 100): 트리의 수
# - subsample(기본값 1): 훈련 샘플 개수의 비율
# - colsample_bytree(기본값 1): 특성 개수의 비율
# - n_jobs(기본값 1): 사용 코어 수 (-1: 모든 코어를 다 사용)
                     )
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9649122807017544

In [12]:
# 조기종료
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss', 
                     learning_rate = 0.05,
                      n_estimators = 500)
eval_set = [(X_test, y_test)] #검증할 데이터를 가져오는 것
model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=10) #학습할 데이터도 가져와야 함
#10번 이상 성능 향상이 없다면 종료하겠다
pred = model.predict(X_test)
accuracy_score(y_test, pred)

[0]	validation_0-logloss:0.65391
[1]	validation_0-logloss:0.61861
[2]	validation_0-logloss:0.58697
[3]	validation_0-logloss:0.55756
[4]	validation_0-logloss:0.53038
[5]	validation_0-logloss:0.50611
[6]	validation_0-logloss:0.48363
[7]	validation_0-logloss:0.46304
[8]	validation_0-logloss:0.44332
[9]	validation_0-logloss:0.42512
[10]	validation_0-logloss:0.40821
[11]	validation_0-logloss:0.39260
[12]	validation_0-logloss:0.37838
[13]	validation_0-logloss:0.36512
[14]	validation_0-logloss:0.35276
[15]	validation_0-logloss:0.34090
[16]	validation_0-logloss:0.33018
[17]	validation_0-logloss:0.31967
[18]	validation_0-logloss:0.30998
[19]	validation_0-logloss:0.30105
[20]	validation_0-logloss:0.29259
[21]	validation_0-logloss:0.28478
[22]	validation_0-logloss:0.27725
[23]	validation_0-logloss:0.27027
[24]	validation_0-logloss:0.26359
[25]	validation_0-logloss:0.25755
[26]	validation_0-logloss:0.25139
[27]	validation_0-logloss:0.24593
[28]	validation_0-logloss:0.24103
[29]	validation_0-loglos

0.9473684210526315

In [14]:
#워닝 없애기

#xgboost
from xgboost import XGBClassifier #불러오기
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)
#성능이 조금이라도 더 올랄감

0.9508771929824561

3-5. 교차검증

교차검증
일반적으로 모델을 학습시킬 때 데이터를 train set과 test set으로 나누어 train set을 가지고 학습을 수행합니다.
교차검증이란 여기서 train set을 다시 train set과 validation set으로 나누어 학습 중 검증과 수정을 수행하는 것을 의미합니다 (각각의 모델 데이터, 실습 데이터로 써서 정확성을 검증)

Kfold
일반적으로 사용되는 교차 검증 기법
StratifiedKfold
불균형한 타겟 비율을 가진 데이터가 한쪽으로 치우치는 것을 방지
사이킷런 교차검증
사이킷런 내부 API를 통해 fit(학습) - predict(예측) - evaluation(평가)

In [15]:
# 데이터셋 로드
def make_dataset2():
    iris = load_breast_cancer()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    return df.drop('target', axis=1), df['target']
X, y = make_dataset2()

In [16]:
# KFold
from sklearn.model_selection import KFold
model = DecisionTreeClassifier(random_state=0)

kfold = KFold(n_splits=5)
for train_idx, test_idx in kfold.split(X): #하나씩 인덱스로 지정, 검증 데이터로 사용
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(accuracy_score(y_test, pred))

0.8771929824561403
0.9122807017543859
0.9473684210526315
0.9385964912280702
0.8407079646017699


In [17]:
# Stratified Kfold
# StratifiedKfold
# 불균형한 타겟 비율을 가진 데이터가 한쪽으로 치우치는 것을 방지
from sklearn.model_selection import StratifiedKFold
model = DecisionTreeClassifier(random_state=0)

kfold = StratifiedKFold(n_splits=5)
for train_idx, test_idx in kfold.split(X, y): #타겟을 넣어서 불균형 방지
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(accuracy_score(y_test, pred))

0.9035087719298246
0.9210526315789473
0.9122807017543859
0.9473684210526315
0.9026548672566371


In [18]:
# 교차검증
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=3) #모델을 넣어 x data, y data를 비교
scores

array([0.88947368, 0.94210526, 0.86243386])

In [19]:
# 평균 점수
scores.mean()

0.8980042699340944

In [20]:
# 교차검증 Stratified Kfold
kfold = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X, y, cv=kfold)
scores

array([0.90350877, 0.92105263, 0.9122807 , 0.94736842, 0.90265487])

In [21]:
# 평균 점수

scores.mean()

0.9173730787144851

평가 (분류 모델)
정확도 accuracy: 실제 값과 예측값이 일치하는 비율
정밀도 precision: 양성이라고 예측한 값 중 실제 양성인 값의 비율 (암이라고 예측 한 값 중 실제 암)
재현율 recall: 실제 양성 값 중 양성으로 예측한 값의 비율 (암을 암이라고 판단); 정밀도와 재현률은 Trade-off 관계
F1: 정밀도와 재현율의 조화평균
ROC-AUC
ROC: 참 양성 비율(True Positive Rate)에 대한 거짓 양성 비율(False Positive Rate) 곡선
AUC: ROC곡선 면적 아래 (완벽하게 분류되면 AUC가 1임)

In [22]:
# 정확도
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9026548672566371

In [23]:
# 정밀도
from sklearn.metrics import precision_score
precision_score(y_test, pred)

0.9545454545454546

In [24]:
# 재현율
from sklearn.metrics import recall_score
recall_score(y_test, pred)

0.8873239436619719

In [25]:
# f1
from sklearn.metrics import f1_score
f1_score(y_test, pred)

0.9197080291970803

In [26]:
# roc_auc
from sklearn.metrics import roc_auc_score
model = XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict_proba(X_test)
roc_auc_score(y_test, pred[:,1])

0.999664654594232